In [24]:
%matplotlib inline
#from keras.applications.resnet50 import ResNet50
# not using keras version since it cannot have an input size < 197x197.  Might also require 3-channel color.
import keras
import resnet
from glob import glob
import numpy as np
import os
import random
from PIL import Image
import matplotlib.pyplot as plt

In [25]:
#K.image_data_format()
print(keras.backend.image_data_format())
print(keras.backend.image_dim_ordering())

channels_last
tf


In [33]:
train_path = 'data/train3'
train_input_image_names = glob(train_path+'/crop_score*png')
train_input_image_names.sort()
train_input_images = np.stack([np.array(Image.open(fn)) for fn in train_input_image_names])
train_input_images.shape = train_input_images.shape + (1,)
num_train_images,rows,cols,channels = train_input_images.shape
num_train_images,rows,cols,channels

(2553, 120, 32, 1)

In [34]:
valid_path = 'data/valid3'
valid_input_image_names = glob(valid_path+'/crop_score*png')
valid_input_image_names.sort()
valid_input_images = np.stack([np.array(Image.open(fn)) for fn in valid_input_image_names])
valid_input_images.shape = valid_input_images.shape + (1,)
num_valid_images,rows,cols,channels = valid_input_images.shape
num_valid_images,rows,cols,channels

(851, 120, 32, 1)

In [31]:
#model = resnet.ResnetBuilder.build_resnet_18((64, 64, 1), 128) # Total params: 11,444,288
#model = resnet.ResnetBuilder.build_resnet_34((64, 64, 1), 128) # Total params: 21,563,584
#model = resnet.ResnetBuilder.build_resnet_50((64, 64, 1), 128) # Total params: 24,025,920
image_shape = (rows,cols,channels)
output_bins = 128 + 16 # midi values + note length
model = resnet.ResnetBuilder.build_resnet_50(image_shape, output_bins)
# Total params: 24,234,320
# Trainable params: 24,188,880
# Non-trainable params: 45,440

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 1, 120)   0                                            
__________________________________________________________________________________________________
conv2d_219 (Conv2D)             (None, 16, 1, 64)    376384      input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_199 (BatchN (None, 16, 1, 64)    256         conv2d_219[0][0]                 
__________________________________________________________________________________________________
activation_199 (Activation)     (None, 16, 1, 64)    0           batch_normalization_199[0][0]    
__________________________________________________________________________________________________
max_poolin